# Final anlalysis

Now that we have some of the sample we are going to do some biostatistical analysis.

In [1]:
## Libreries
library(tidyverse)

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“package ‘purrr’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”
Warning message:
“package ‘stringr’ was built under R version 4.2.3”
Warning message:
“package ‘forcats’ was built under R version 4.2.3”
Warning message:
“package ‘lubridate’ was built under R version 4.2.3”
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted packa

In [3]:
## reading the data ##

# Biotype
apc_biotype <- read_tsv("../results/results_workflow/chromosome5/biotype.tsv") %>%
    mutate(gene="APC") 
braf_biotype <- read_tsv("../results/results_workflow/chromosome7/biotype.tsv") %>%
    mutate(gene="BRAF")
kras_biotype <- read_tsv("../results/results_workflow/chromosome12/biotype.tsv") %>%
    mutate(gene="KRAS")
tp53_biotype <- read_tsv("../results/results_workflow/chromosome17/biotype.tsv") %>%
    mutate(gene="TP53")

Rows: 30 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): biotype, sample
dbl (1): n

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 41 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): biotype, sample
dbl (1): n

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 33 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): biotype, sample
dbl (1): n

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types 

In [107]:
## Joining the data
biotype <- rbind(apc_biotype,braf_biotype,tp53_biotype,kras_biotype)  

# A tibble: 148 × 4
    biotype                            n sample          gene 
    <chr>                          <dbl> <chr>           <chr>
  1 protein_coding                    45 ERR753368_chr5  APC  
  2 nonsense_mediated_decay           17 ERR753368_chr5  APC  
  3 retained_intron                    2 ERR753368_chr5  APC  
  4 protein_coding                    49 ERR753369_chr5  APC  
  5 nonsense_mediated_decay           18 ERR753369_chr5  APC  
  6 retained_intron                    2 ERR753369_chr5  APC  
  7 protein_coding                    61 ERR753370_chr5  APC  
  8 nonsense_mediated_decay           22 ERR753370_chr5  APC  
  9 retained_intron                    3 ERR753370_chr5  APC  
 10 protein_coding                    45 ERR753372_chr5  APC  
 11 nonsense_mediated_decay           17 ERR753372_chr5  APC  
 12 retained_intron                    2 ERR753372_chr5  APC  
 13 protein_coding                    69 ERR753373_chr5  APC  
 14 nonsense_mediated_decay        

In [109]:
## Processing the data
df_biotype <- expand.grid(biotype=as.character(unique(as.character(biotype$biotype))),
                          sample=as.character(unique(as.character(biotype$sample))),
                          gene=as.character(unique(as.character(biotype$gene)))) %>% 
    left_join(
        biotype, by=c("biotype","sample","gene")
        ) %>% 
    mutate(n=ifelse(is.na(n),0,n)) %>%
    separate_wider_delim(sample,delim = "_", names = c("sample", "chr")) %>%
    select(-"chr") 


In [122]:
## Ploting ggplot2 biotype-genes
biotype_plot <- df_biotype %>%
    group_by(gene,biotype) %>%
    summarise(n=sum(n)) %>%
    mutate(biotype=factor(biotype,
                          levels = c("protein_coding", "protein_coding_CDS_not_defined", 
                                     "nonsense_mediated_decay", "retained_intron"),
                          labels = c("Portein coding", "Protein coding (not define)",
                                     "Nonsense mediated decay", "Retained intron"))) %>%
    ggplot(aes(reorder(gene,n),n, fill=biotype)) +
    geom_bar(stat="identity",position=position_dodge(.65), width=.5,
             color="black") +
    scale_y_continuous(expand=expansion(0),
                       limits=c(0,270),
                       breaks=seq(0,250,25)) +
    scale_fill_manual(values=c("darkgray","red", "blue", "forestgreen")) +
    labs(
        title="Variant biotypes per genes",
        x="Samples",
        y="Number of variants",
        fill="Biotype"
    ) +
    theme_classic() +
    theme(
        plot.title=element_text(hjust=.5, face="bold", size=14),
        axis.title=element_text(face="bold", size=12),
        axis.text=element_text(color="black", size=10),
        axis.ticks=element_blank(),
        legend.position=c(.3,.8),
        legend.background=element_rect(color="black")
    )


ggsave(file = "../results_workflow/plots_workflow/biotype.png",
       plot = biotype_plot,
       width = 7,
       heigh = 5)

`summarise()` has grouped output by 'gene'. You can override using the `.groups` argument.
Warning message:
“Removed 2 rows containing missing values (`geom_bar()`).”


In [89]:
?pivot_wider()